In [3]:
import happybase
import sys
sys.path.append('/root/face/facenet')
sys.path.append('/root/face/facenet/src')
import dnet_face
import cv2
import numpy as np
import os
import pickle
import tensorflow as tf
import random
from sklearn.svm import SVC

In [4]:
HBASE_HOST = '192.168.1.91'
HBASE_PORT = 9090
HBASE_TABLE = 'feature'

CLASS_NAMES_DICT = {}
CLASS_LIST = []

CLASSIFIER_NAME_INP = '/root/face/facenet/pkl/add_100.pkl'
CLASSIFIER_NAME_EXP = '/root/face/facenet/pkl/test.pkl'

INPUT_IMAGE_DIR = '/root/face/image/add20'

GPU_MEMORY_FRACTION = 0.2

In [24]:
class FeatureDB:
    
    def __init__(self):
        with open(CLASSIFIER_NAME_INP, 'rb') as f:
            CLASS_LIST = pickle.load(f)[1]
            for i in range(len(CLASS_LIST)):
                CLASS_NAMES_DICT[CLASS_LIST[i]] = i
        
        self.face = dnet_face.FACE()
                
    def load_image(self):
        
        image_list = []
        
        image_dir_list = os.listdir(INPUT_IMAGE_DIR)
        
        for directory in image_dir_list:
            image_dir = os.path.join(INPUT_IMAGE_DIR, directory)
            image_file_list = os.listdir(image_dir)
            
            for image_file in image_file_list:
                image_path = os.path.join(image_dir, image_file)
                image_list.append(image_path)
            
        return image_list
    
    def hbase_connection(self):
        try:
            conn = happybase.Connection(HBASE_HOST, port=HBASE_PORT)
            table = conn.table(HBASE_TABLE)
            
            return table
        
        except Exception as e:
            print (e)
            
    def hbase_put(self, table, result, emb):
        
        face_id = result[0]['id']
        face_img = result[0]['img']
        face_vec = emb.tobytes()
        
        try:
            table.put(face_id + str(random.randrange(0, 100000)),{
                b'feature_cf:vec':face_vec,
                b'id_cf:id':face_id
            })
            
        except Exception as e:
            print (e)
        
    def hbase_get(self, table):
        
        emb_list = []
        labels = []
        
        for rowkey, value in table.scan():
            emb = np.frombuffer(value['feature_cf:vec'], dtype=np.float32)
            emb = np.expand_dims(emb, axis=0)
            emb_list.append(emb)
            labels.append(CLASS_NAMES_DICT[value['id_cf:id']])
    
        emb_array = np.asarray(emb_list)
        emb_array = np.squeeze(emb_array, axis=1)
        
        return emb_array, labels
    
    def train_classifier(self):
        
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=GPU_MEMORY_FRACTION)
        config = tf.ConfigProto(gpu_options=gpu_options)
        
        table = self.hbase_connection()
        emb_array, labels = self.hbase_get(table)
        
        print ('{} face feature vectors and {} labels loaded'.format(emb_array.shape[0], len(labels)))
        
        with tf.Session(config=config) as sess:
            model = SVC(kernel='linear', probability=True)
            model.fit(emb_array, labels)
        
        with open(CLASSIFIER_NAME_EXP, 'wb') as outfile:
            pickle.dump((model, CLASS_LIST), outfile)
            print('Saved classifier model to file "%s"' % CLASSIFIER_NAME_EXP)
            

    def inference_and_put(self):
        image_list = self.load_image()
        table = self.hbase_connection()
        
        for image_file in image_list:
            img = cv2.imread(image_file)
            result, prob = self.face.classification(img)
            emb = self.face.emb[0]
            
            self.hbase_put(table, result, emb)
            
        print ('Inference complete : {} files'.format(len(image_list)))
            
        


In [25]:
fdb = FeatureDB()

Loading feature extraction Model....
Model filename: /root/face/facenet/models/20170512.pb
Model Loaded : 3.77759981155
Loaded classifier from file "/root/face/facenet/pkl/add_100.pkl"


In [16]:
fdb.inference_and_put()

Inference complete : 100 files


In [26]:
fdb.train_classifier()

399 face feature vectors and 399 labels loaded
Saved classifier model to file "/root/face/facenet/pkl/test.pkl"
